# L08.1: Similarity Practice

    -본 실습 코드는 kaggle을 통해서 진행해야함

In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 데이터 불러오기

In [20]:
meta = pd.read_csv( '/kaggle/input/the-movies-dataset/movies_metadata.csv' )
keywords = pd.read_csv( '/kaggle/input/the-movies-dataset/keywords.csv' )
ratings = pd.read_csv( '/kaggle/input/the-movies-dataset/ratings_small.csv' )
display(meta, keywords, ratings)

#### meta

In [21]:
meta = meta[ ['id', 'original_title', 'original_language', 'genres'] ] #meta에서 특정컬럼만 불러옴
meta = meta.rename(columns={'id':'movieId','original_title': 'title', 'original_language': 'language'}) #컬럼 이름 변경
meta = meta.loc[meta['language'] == 'en',:]#lamguage가 en인 행만 추출
meta.movieId = pd.to_numeric(meta.movieId) #movieid를 영어로 변경
meta.movieId

In [22]:
#json srt을 set로 변경
def str_to_set(x):
    genre_set = set()
    for item in eval(x):
        genre_set.add(item['name'])
    return genre_set

In [23]:
#meta의 genres컬럼에 함수를 적용
meta.genres = meta.genres.apply(str_to_set);meta

#### keywords

In [24]:
keywords

In [25]:
#위에서 정의한 srt_ot_set함수 동일하게 적용
keywords.keywords = keywords.keywords.apply(str_to_set);keywords

In [26]:
keywords = keywords.rename(columns={'id': 'movieId'}) #컬럼이름 변경
keywords.movieId = pd.to_numeric(keywords.movieId);keywords #숫자로 타입변경

#### meta, keywords 병합

In [27]:
#meta, keywords를 meta에 병합
meta = pd.merge(meta, keywords, on='movieId', how='inner');meta

In [28]:
dk = meta.loc[meta.title == 'The Dark Knight'].iloc[0] #다크나이트 영화중 첫번째 나오는 것을 dk로 지정
dkr = meta.loc[meta.title == 'The Dark Knight Rises'].iloc[0] #위와 동일
pd.concat([dk, dkr], axis=1).T #병합 axis=1 -> 열방향르오 작동

In [29]:
#자카드 유사도를 구하는 함수를 생성
def jaccard_similarity(s1, s2):
    if len(s1|s2) == 0:
        return 0
    return len(s1&s2)/len(s1|s2)

In [30]:
#dk와 dkr의 자카드 유사도를 구함
jaccard_similarity(dk.genres|dk.keywords, dkr.genres|dkr.keywords)

#### ratings

In [31]:
ratings

In [32]:
ratings.movieId = pd.to_numeric(ratings.movieId)

In [36]:
#ratings와 meta의 특정 컬럼을 병합
ratings = pd.merge(ratings, meta[['movieId', 'title']], on='movieId', how='inner')

In [37]:
#피벗테이블을 matrix라는 이름으로 지정
matrix = ratings.pivot_table(index= 'userId', columns='title', values='rating');matrix

In [38]:
#피어슨 상관계수 구하는 코드 작성
def pearson_similarity(u1, u2):
    u1_c = u1 - u1.mean()
    u2_c = u2 - u2.mean()
    denom = np.sqrt(np.sum(u1_c ** 2) * np.sum(u2_c ** 2))
    if denom != 0:
        return np.sum(u1_c * u2_c)/denom
    else:
        return 0

In [39]:
#피어슨 상관계를 구함
dk_rating = matrix['The Dark Knight']
pk_rating = matrix['Prom Night']
pearson_similarity(dk_rating, pk_rating)

In [40]:
#비슷한 영화를 찾아주는 코드를 작성
def find_similar_movies (input_title , matrix, n, alpha):
    input_meta = meta.loc[ meta[ 'title'] == input_title].iloc[ 0]
    input_set = input_meta.genres | input_meta.keywords

    result = []

    for this_title in matrix.columns:
        if this_title == input_title:
            continue
        this_meta = meta.loc[ meta[ 'title'] == this_title].iloc[ 0]
        this_set = this_meta.genres | this_meta.keywords

        pearson = pearson_similarity(matrix[this_title], matrix[input_title])
        jaccard = jaccard_similarity(this_set, input_set)

        score = alpha * pearson + ( 1-alpha) * jaccard
        result.append( (this_title, pearson, jaccard, score) )

    result.sort(key= lambda r: r[3], reverse= True)

    return result[:n]

In [42]:
result = find_similar_movies('The Dark Knight', matrix, 10, 0.3)
pd.DataFrame(result, columns = ['title', 'pearson', 'jaccard', 'score'])

# END